Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Model Development with Custom Weights

This example shows how to retrain a model with custom weights and fine-tune the model with quantization, then deploy the model running on FPGA. Only Windows is supported. We use TensorFlow and Keras to build our model. We are going to use transfer learning, with ResNet50 as a featurizer. We don't use the last layer of ResNet50 in this case and instead add our own classification layer using Keras.

The custom wegiths are trained with ImageNet on ResNet50. We are using a public Top tagging dataset as our training data.

Please set up your environment as described in the [quick start](project-brainwave-quickstart.ipynb).

This work was performed on the Caltech GPU cluster. The specific server is named imperium-sm.hep.caltech.edu. Paths have been set to work in that environment, but must be altered for your purposes.

In [1]:
import os,sys
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
import numpy as np
from keras import backend as K
import tables
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Using TensorFlow backend.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3724801581329058975, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15578061210
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7446580128410452136
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"]

## Setup Environment
After you train your model in float32, you'll write the weights to a place on disk. We also need a location to store the models that get downloaded.

In [2]:
# These directories were chosen because they write the data to local disk, which will have the fastest access time
# of our various storage options.
custom_weights_dir = os.path.expanduser("../weights-floatingpoint")
custom_weights_dir_q = os.path.expanduser("../weights-quantized")
custom_weights_dir_tl = os.path.expanduser("../weights-transferlearning-floatingpoint")
saved_model_dir = os.path.expanduser("../models")
resutls_dir = os.path.expanduser("../results")

## Prepare Data
Load the files we are going to use for training and testing. The public Top dataset consists of image formatted data, but our data has been preprocessed into a raw form.

At the time of writing, the files in question are located at `/data/shared/dwerran/converted`. They are stored in the HDF5 format, and must be accessed via the `tables` module. The two sub-datasets we're interested in are `/img-pt` and `/labels`, corresponding to the images and lables respectively. Each dataset contains 50000 images, and there are about 30 datasets. As before, this storage location was chosen to maximize data bandwidth.

In [4]:
from utils import normalize_and_rgb, image_with_label, count_events, save_results, plot_results

In [5]:
import glob
datadir = "../data/"
n_train_file = 25
n_test_file = 9
n_val_file = 9

train_files = glob.glob(os.path.join(datadir, 'train_file_*'))
test_files = glob.glob(os.path.join(datadir, 'test/test_file_*'))
val_files = glob.glob(os.path.join(datadir, 'val_file_*'))
train_files = train_files[:n_train_file]
test_files = test_files[:n_test_file]
val_files = test_files[:n_val_file]

n_train_events = count_events(train_files)
n_test_events = count_events(test_files)
n_val_events = count_events(val_files)

print("n_train_events =", n_train_events)
print("n_test_events =", n_test_events)
print("n_val_events =", n_val_events)

n_train_events = 1211000
n_test_events = 404000
n_val_events = 404000


## Construct Model
We use ResNet50 for the featuirzer and build our own classifier using Keras layers. We train the featurizer and the classifier as one model. The weights trained on ImageNet are used as the starting point for the retraining of our featurizer. The weights are loaded from tensorflow checkpoint files.

Before passing image dataset to the ResNet50 featurizer, we need to preprocess the input file to get it into the form expected by ResNet50. ResNet50 expects float tensors representing the images in BGR, channel last order. Given that our images are greyscale, this isn't relevant to us, as we will simply be copying the data in place.

In [6]:
from utils import preprocess_images

We use Keras layer APIs to construct the classifier. Because we're using the tensorflow backend, we can train this classifier in one session with our Resnet50 model.

In [7]:
from utils import construct_classifier

Now every component of the model is defined, we can construct the model. Constructing the model with the project brainwave models is two steps - first we import the graph definition, then we restore the weights of the model into a tensorflow session. Because the quantized graph defintion and the float32 graph defintion share the same node names in the graph definitions, we can initally train the weights in float32, and then reload them with the quantized operations (which take longer) to fine-tune the model.

In [8]:
from utils import construct_model

## Train Model
First we train the model with custom weights but without quantization. Training is done with native float precision (32-bit floats). We load the traing data set and batch the training with 10 epochs. When the performance reaches desired level or starts decredation, we stop the training iteration and save the weights as tensorflow checkpoint files. 

In [9]:
from utils import chunks, train_model, test_model

This training currently leverages a hack to work around some apparent limits in the BW API. I have attempted to specify a custom weights directory when calling the `Resnet50` function in `construct_model()` above in the same way it is specified for `Quantized_Resnet50`. However, this throws an error, and since there is no API documentation yet, the way I'm working around it is rewriting our trained weights to the saved model directory. I will be reaching out to the team on this topic to see if they have a better suggestion.

In [10]:
# Launch the training
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

num_epoch_train = 0

with sess.as_default():
    in_images, image_tensors, features, preds, featurizer, classifier = construct_model(quantized=False, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir, is_training=True)
    saver = tf.train.Saver(max_to_keep = 100)
    loss_over_epoch, accuracy_over_epoch, auc_over_epoch, val_loss_over_epoch, val_accuracy_over_epoch, val_auc_over_epoch = \
        train_model(preds, in_images, train_files, val_files, is_retrain=True, train_epoch=num_epoch_train, 
                    classifier=classifier,
                    saver=saver, checkpoint_path=custom_weights_dir) 
    _, _, features, preds, featurizer, classifier = construct_model(quantized=False, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir, is_training=False)
    loss, accuracy, auc, preds_test, test_labels = test_model(preds, in_images, test_files)

in restore_weights
checkpoint directory: ../weights-floatingpoint
lastest checkpoint: ../weights-floatingpoint/resnet50_bw_best
INFO:tensorflow:Restoring parameters from ../weights-floatingpoint/resnet50_bw_best
loading classifier weights from ../weights-floatingpoint/class_weights.h5
in restore_weights
checkpoint directory: ../weights-floatingpoint
lastest checkpoint: ../weights-floatingpoint/resnet50_bw_best
INFO:tensorflow:Restoring parameters from ../weights-floatingpoint/resnet50_bw_best
loading classifier weights from ../weights-floatingpoint/class_weights.h5


6319it [08:22, 12.57it/s]                            


test_loss =  0.172
Test Accuracy: 0.931 , Area under ROC curve: 0.982


## Load and Test Model
Here, we re-load the weights saved on disk and test the model.

In [13]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

with sess.as_default():
    print("Loading a trained model")
    in_images, image_tensors, features, preds, featurizer, classifier = construct_model(quantized=False, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir, is_training=False)
    loss, accuracy, auc, preds_test, test_labels = test_model(preds, in_images, test_files)
    print("Accuracy:", accuracy, ", Area under ROC curve:", auc)
    
save_results(results_dir, 't', test_labels, preds_test)

Loading a trained model
in restore_weights
checkpoint directory: ../weights-floatingpoint
lastest checkpoint: ../weights-floatingpoint/resnet50_bw_best
INFO:tensorflow:Restoring parameters from ../weights-floatingpoint/resnet50_bw_best
loading classifier weights from ../weights-floatingpoint/class_weights.h5


6319it [07:45, 13.56it/s]                            


test_loss =  0.582
Test Accuracy: 0.648 , Area under ROC curve: 0.788
Accuracy: 0.6475099009900461 , Area under ROC curve: 0.7875706250289864


NameError: name 'save_results' is not defined

## Test Quantized Model
After training, we evaluate the trained model's accuracy on test dataset with quantization. So that we know the model's performance if it is deployed on the FPGA. The only significant difference between this cell and the cell two below is this loads weights from the floating point directory and the one two below loads from the quantized directory. In this way, you can compare pre- and post- quantization fine tuning tests.

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

with sess.as_default():
    print("Testing trained model with quantization")
    in_images_q, image_tensors_q, features_q, preds_q, quantized_featurizer, classifier = construct_model(quantized=True, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir, is_training=False)
    loss_q, accuracy_q, auc_q, preds_test_q, test_labels_q = test_model(preds_q, in_images_q, test_files)
    print("Accuracy:", accuracy_q, ", Area under ROC curve:", auc_q)
    
save_results(results_dir, 'q' test_labels_q, preds_test_q)

## Fine-Tune Quantized Model
Sometimes, the model's accuracy can drop significantly after quantization. In those cases, we need to retrain the model enabled with quantization to get better model accuracy.

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

num_epoch_finetune = 1

with sess.as_default():
    print("Fine-tuning model with quantization")        
    
    in_images, image_tensors, features, preds, quantized_featurizer, classifier = construct_model(quantized=True, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir, is_training=True)
    saver = tf.train.Saver(max_to_keep = 100)
    loss_over_epoch_ft, accuracy_over_epoch_ft, auc_over_epoch_ft, val_loss_over_epoch_ft, val_accuracy_over_epoch_ft, val_auc_over_epoch_ft = \
        train_model(preds, in_images, train_files, val_files, is_retrain=True, train_epoch=num_epoch_finetune, 
                    classifier=classifier,
                    saver=saver, checkpoint_path=custom_weights_dir_q) 
    loss_ft, accuracy_ft, auc_ft, preds_test_ft, test_labels_ft = test_model(preds, in_images, test_files)

## Load and Test Quantized Model
After training, we evaluate the trained model's accuracy on test dataset with quantization. So that we know the model's performance if it is deployed on the FPGA. The only significant difference between this cell and the cell two above is this loads weights from the quantized directory and the one two above loads from the floating point directory. In this way, you can compare pre- and post- quantization fine tuning tests.

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())
with sess.as_default():
    print("Re-load fine-tuned model with quantization")   
    in_images_q, image_tensors_q, features_q, preds_q, quantized_featurizer, classifier = construct_model(quantized=True, saved_model_dir=saved_model_dir, starting_weights_directory=custom_weights_dir_q, is_training=False)
    loss_ft, accuracy_ft, auc_ft, preds_test_ft, test_labels_ft = test_model(preds_q, in_images_q, test_files)

## Transfer Learning Training

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

num_epoch_train = 20

with sess.as_default():
    in_images, image_tensors, features, preds, featurizer, classifier = construct_model(quantized=False, is_frozen=True, saved_model_dir=saved_model_dir)
    saver = tf.train.Saver(max_to_keep = 100)
    loss_over_epoch_tl, accuracy_over_epoch_tl, auc_over_epoch_tl, val_loss_over_epoch_tl, val_accuracy_over_epoch_tl, val_auc_over_epoch_tl = \
        train_model(preds, in_images, train_files, val_files, is_retrain=True, train_epoch=num_epoch_train, 
                    classifier=classifier,
                    saver=saver, checkpoint_path=custom_weights_dir_tl) 
    loss_tl, accuracy_tl, auc_tl, preds_test_tl, test_labels_tl = test_model(preds, in_images, test_files)

## Load and Test Transfer Learning Model

In [ ]:
# Load a training
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

with sess.as_default():
    print("Loading a trained model")
    in_images, image_tensors, features, preds, featurizer, classifier = construct_model(quantized=False, is_frozen=True, saved_model_dir=saved_model_dir)
    print("loading classifier weights from", custom_weights_dir_tl+'/class_weights.h5')
    classifier.load_weights(custom_weights_dir_tl+'/class_weights.h5')
    loss_tl, accuracy_tl, auc_tl, preds_test_tl, test_labels_tl = test_model(preds, in_images, test_files)
    print("Accuracy:", accuracy_tl, ", Area under ROC curve:", auc_tl)

## Load and Test Quantized Transfer Learning Model

In [ ]:
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

with sess.as_default():
    print("Re-load fine-tuned model with quantization")   
    in_images_q, image_tensors_q, features_q, preds_q, quantized_featurizer, classifier = construct_model(quantized=True, is_frozen=True, saved_model_dir=saved_model_dir)
    print("loading classifier weights from", custom_weights_dir_tl+'/class_weights.h5')
    classifier.load_weights(custom_weights_dir_tl+'/class_weights.h5')
    loss_tl_q, accuracy_tl_q, auc_tl_q, preds_test_tl_q, test_labels_tl_q = test_model(preds_q, in_images_q, test_files)

In [ ]:
plot_results(results_dir)

## Appendix

License for plot_confusion_matrix:

New BSD License

Copyright (c) 2007-2018 The scikit-learn developers.
All rights reserved.


Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

  a. Redistributions of source code must retain the above copyright notice,
     this list of conditions and the following disclaimer.
  b. Redistributions in binary form must reproduce the above copyright
     notice, this list of conditions and the following disclaimer in the
     documentation and/or other materials provided with the distribution.
  c. Neither the name of the Scikit-learn Developers  nor the names of
     its contributors may be used to endorse or promote products
     derived from this software without specific prior written
     permission. 


THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
ARE DISCLAIMED. IN NO EVENT SHALL THE REGENTS OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT
LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY
OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH
DAMAGE.
